In [1]:
import findspark
import os
findspark.init()

from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("deduplication") \
    .master("local[4]") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
    .getOrCreate()

spark.sparkContext.setCheckpointDir("C:/Users/MattM/AppData/Local/Temp/")

#os.environ['HADOOP_HOME'] = "C:/Users/MattM/hadoop/hadoop-3.3.5/bin"
os.environ['HADOOP_HOME'] = "/"
os.environ['HADOOP_MAPRED_HOME'] = "/"


In [2]:
from create_synthetic_records import create_synthetic_records

num_records = 10
num_duplicates0 = 5
num_duplicates1 = 3
num_duplicates2 = 2
num_duplicates3 = 0

df = spark.createDataFrame(create_synthetic_records(num_records, num_duplicates0, num_duplicates1, num_duplicates2, num_duplicates3))

df.show()
df.count()

c:\Users\MattM\Documents\GitHub\string-cluster-algos\create_synthetic_records.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(duplicate, ignore_index=True)
c:\Users\MattM\Documents\GitHub\string-cluster-algos\create_synthetic_records.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(duplicate, ignore_index=True)
c:\Users\MattM\Documents\GitHub\string-cluster-algos\create_synthetic_records.py:151: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(duplicate, ignore_index=True)


+---------+--------------------+--------------------+--------------------+--------------------+------------+-----------------+
|client_id|         client_name|             address|               email|        phone_number|duplicate_id|num_modifications|
+---------+--------------------+--------------------+--------------------+--------------------+------------+-----------------+
|       13|  Jennifer Gallagher|85697 Taylor Land...|acevedosusan@exam...|  416.388.2314x56779|           1|                0|
|       10|     Carlos Peterson|512 Kathryn Flats...|   amy23@example.net|    197-751-6296x336|          10|                0|
|       18|      Charles Barker|4704 Hall Parkway...|erincampbell@exam...|        728-800-8405|           3|                1|
|       20|Christopher Rober...|0448 Julie Garden...|bowenfrank@exampl...|          4823355590|           5|                0|
|       19|        David Nguyen|USCGC Haley, FPO ...|adrienne45@hotmai...|   365.484.6647x0370|           4|   

20

In [3]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Tokenizer, SQLTransformer, RegexTokenizer, NGram, HashingTF
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

from pyspark.sql.functions import col, concat_ws, split, array_sort, min
from pyspark.sql.window import Window
from pyspark.ml.feature import MinHashLSH

from graphframes import *
#parameters
distance_threshold = 0.1

#black box
transform0  =    SQLTransformer(statement="""SELECT *,LOWER(REGEXP_REPLACE(CONCAT(client_name, address, email, phone_number), 
        '[\\s\\W]', '')) AS record_strings FROM __THIS__ """)
token0      =    Tokenizer(inputCol="client_name", outputCol="token" )
transform1  =    SQLTransformer(statement="SELECT *, concat_ws(' ', token) concat FROM __THIS__")
token1      =    RegexTokenizer(pattern="", inputCol="concat", outputCol="char", minTokenLength=1 )
ngram       =    NGram(n=2, inputCol="char", outputCol="ngram")
hash      =    HashingTF(inputCol="ngram", outputCol="vector")

# add to pipeline when using big data
feat        =    VectorAssembler(inputCols=["vector"], outputCol="features")
kmeans      =    KMeans(k = 2, seed = 1, predictionCol="kmeans")

stages = [transform0,token0,transform1,token1,ngram,hash] #feat,kmeans

#pre-processing
pipeline = Pipeline(stages=stages)
model = pipeline.fit(df)
model_df = model.transform(df)

 
#knn then jaccard distance
lsh_model = MinHashLSH(inputCol="vector", outputCol="lsh", numHashTables=3).fit(model_df)
similarity_df = lsh_model.approxSimilarityJoin(model_df, model_df, distance_threshold, distCol="text_distance").filter("datasetA.client_id != datasetB.client_id")

 
#graphx
edges = (similarity_df.selectExpr("datasetA.client_id as src","datasetB.client_id as dst")
        .withColumn('set_col', concat_ws(',', col('src'), col('dst')))
        .withColumn('sorted_set', array_sort(split(col('set_col'), ',')))
        .dropDuplicates(['sorted_set']).select(col("src"), col("dst")))

vertices = (similarity_df.selectExpr("datasetA.client_id as id").union(similarity_df.selectExpr("datasetB.client_id as id"))).distinct()


#connections graph
graph_frame = GraphFrame(vertices, edges)

#slow
components_df = graph_frame.connectedComponents().withColumn("min_id", min(col("id")).over(Window.partitionBy("component")))

c:\Users\MattM\anaconda3\envs\spark\lib\site-packages\pyspark\sql\dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling o218.run.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputCommitt